In [2]:
import requests
import os
import pandas as pd
from datetime import datetime
from tqdm import tqdm

### 1. Articles scrapping

In [39]:
with open('../.api_auth') as f:
    (api_login, api_pass) = [line.rstrip() for line in f]

ids = []
leads = []
texts = []

date_range = pd.date_range(start="2023-01-01", end="2023-01-31").strftime('%d.%m.%Y').to_list()
for date in tqdm(date_range):
    r1 = requests.get(f'https://api.sta.si/news/sl/{date}', auth=(api_login, api_pass))
    articles_id = eval(r1.content)

    for id in articles_id:
        r2 = requests.get(f'https://api.sta.si/news/sta/{id}', auth=(api_login, api_pass))
        ids.append(id)
        leads.append(r2.json().get('lede', ''))
        texts.append(r2.json().get('text', ''))

dataset = pd.DataFrame({'id': ids, 'lead': leads, 'text': texts})
dataset.to_json(f'../data/raw_articles/01_2023.jsonl', lines=True, orient='records')

100%|██████████| 31/31 [11:34<00:00, 22.41s/it]


### 2. Preparing evaluation dataset

In [12]:
# df = pd.read_json("..\\data\\raw_articles\\all_2023.jsonl", lines=True)
# df_eval = df.sample(100).reset_index(drop=True)
# df_eval.to_json(f'../data/raw_articles/all_2023_eval.jsonl', lines=True, orient='records', force_ascii=False)

In [48]:
# df = pd.read_json("..\\data\\raw_annotated\\sta_01.json", lines=True)['examples']

# id      = []
# content = []
# start   = []
# end     = []
# value   = []

# for i in range(len(df[0])):
#     row = df[0][i]
#     for annotation in row['annotations']:
#         if annotation['correct'] == True:
#             id.append(row['metadata']['id'])
#             content.append(row['content'])
#             start.append(annotation['start'])
#             end.append(annotation['end'])
#             value.append(annotation['value'])


In [67]:
# eval_df = pd.DataFrame({'id': id, 'text': content, 'entity': value, 'start': start, 'end': end})
# eval_df.to_json(f'../data/2023_eval.jsonl', lines=True, orient='records', force_ascii=False)

### 3. SlovenianNER

In [5]:
eval_df = pd.read_json("..\\data\\2023_eval.jsonl", lines=True)
eval_df

,id,text,entity,start,end
0,3150358,Slovenska javnost prispevku dobrega zdravja in...,Lana Blinc,523,533
1,3161463,"Naravnanost fiskalne politike, predvidena v sr...",fiskalni svet,262,275
2,3151733,S francoskim filmom Črta režiserke Ursule Meie...,Kinodvoru,62,71
3,3151733,S francoskim filmom Črta režiserke Ursule Meie...,Francoskem inštitutu v Sloveniji,2290,2322
4,3152968,"Kitajska vojska je danes sporočila, da je opoz...",francoska tiskovna agencija AFP,691,722
...,...,...,...,...,...
87,3158301,Andreja Kutin v komentarju Zelena prehajanja o...,Andreja Kutin,0,13
88,3158725,Hokejisti Sija Acronija Jesenic bodo v soboto ...,Gašper Glavič,1183,1196
89,3158725,Hokejisti Sija Acronija Jesenic bodo v soboto ...,Gaber Glavič,1888,1900
90,3150559,Eden največjih svetovnih proizvajalcev avtobus...,Volvo Buses,49,60
